<a href="https://colab.research.google.com/github/Mario-U-KY/DataScience/blob/main/3_3_Concise_Implementation_of_Linear_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

3_2에서의 과정을 API 사용으로 간단히 해보자

In [1]:
pip install d2l

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


<h2>3_3_1 Generating the Dataset

In [4]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

In [6]:
true_w  = torch.tensor([2,-3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

<h2>3_3_2 Reading the Dataset

API 로 데이터 읽어오기!

In [9]:
def load_array(data_arrays, batch_size, is_train=True):
  dataset = data.TensorDataset(*data_arrays)
  return data.DataLoader(dataset,batch_size,shuffle=is_train)

In [10]:
batch_size = 10
data_iter = load_array((features, labels),batch_size)

In [11]:
next(iter(data_iter))

[tensor([[ 1.8318,  0.8830],
         [-1.5078,  1.0042],
         [-0.4905, -0.0497],
         [ 0.4997, -1.2484],
         [-0.1964,  0.0955],
         [-0.0399, -0.0127],
         [ 0.1893,  0.8553],
         [-0.9945, -1.3140],
         [ 0.8268, -1.6280],
         [ 1.4553,  0.7856]]), tensor([[ 4.8540],
         [-2.2203],
         [ 3.3755],
         [ 9.4450],
         [ 3.4810],
         [ 4.1668],
         [ 1.6571],
         [ 6.6708],
         [11.3857],
         [ 4.4323]])]

3_3_3 Defining the Model

In [12]:
from torch import nn
net = nn.Sequential(nn.Linear(2,1))

<h2>3_3_4 Initializing model Parameters

weight parameter를 평균이 0이고 std 0.01인 정규분포로 부터 뽑아온다. bias는 처음에 0으로 설정된다.

In [13]:
#첫번째 레이어를 지정할 수 있다. net[0]로.
net[0].weight.data.normal_(0,0.01)
net[0].bias.data.fill_(0)

tensor([0.])

<h2>3_3_5 Defining the Loss Function

In [14]:
loss = nn.MSELoss()

<h2>3_3_6 Defining the Optimization Algorithm

In [15]:
trainer = torch.optim.SGD(net.parameters(),lr=0.03)

<h2>3_3_7 Training

Training 과정은 3가지 단계로 구성됨.
1. net(X)로 prediction 진행, loss 구하기
2. backpropagation으로 gradients 구하기
3. optimizer 사용해서 파라미터 업데이트

In [18]:
num_epochs = 3
for epoch in range(num_epochs):
  for X, y in data_iter:
    l = loss(net(X),y)
    trainer.zero_grad()
    l.backward()
    trainer.step()
  l = loss(net(features),labels)
  print(f'epoch {epoch+1}, loss {l:f}')

epoch 1, loss 0.000092
epoch 2, loss 0.000092
epoch 3, loss 0.000091


In [19]:
w = net[0].weight.data
print('error in estimating w:', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('error in estimating b:', true_b - b)

error in estimating w: tensor([4.5419e-05, 2.5105e-04])
error in estimating b: tensor([0.0002])
